# Import packages

- Remember the guys to git pull tomorrow morning
- tensorflow was added in the requirements.txt

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Import data 

In [2]:
data = pd.read_csv('../raw_data/fer2013.csv')
data.columns

Index(['emotion', 'pixels', 'Usage'], dtype='object')

#

In [3]:
train_set = data[(data.Usage == 'Training')] 
val_set = data[(data.Usage == 'PublicTest')]
test_set = data[(data.Usage == 'PrivateTest')] 

In [4]:
# define X_train, val, test

X_train = np.array(list(map(str.split, train_set.pixels)), np.float32) 
X_val = np.array(list(map(str.split, val_set.pixels)), np.float32) 
X_test = np.array(list(map(str.split, test_set.pixels)), np.float32) 

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1) 
X_val = X_val.reshape(X_val.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [5]:
# Define y_train,  val, test
y_train =  train_set.emotion
y_val = val_set.emotion
y_test = test_set.emotion


In [6]:
X_train = (X_train/255) -0.5
X_val = (X_val/255) - 0.5
X_test = (X_test/255) - 0.5

In [7]:
X_train = X_train.reshape(X_train.shape[0], 48, 48) 
X_val = X_val.reshape(X_val.shape[0], 48, 48)
X_test = X_test.reshape(X_test.shape[0], 48, 48)

X_train_rgb = np.repeat(X_train[..., np.newaxis], 3, -1)
X_val_rgb = np.repeat(X_val[..., np.newaxis], 3, -1)
X_val_rgb.shape

(3589, 48, 48, 3)

In [8]:
y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)
y_test_cat = to_categorical(y_test)

y_train_cat.shape

(28709, 7)

# Model architeture

In [9]:
def plot_loss_accuracy(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    
    # --- LOSS --- 
    
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylim((0,3))
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- ACCURACY
    
    ax[1].plot(history.history['accuracy'])
    ax[1].plot(history.history['val_accuracy'])
    ax[1].set_title('Model Accuracy')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].set_ylim((0,1))
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
    
    if title:
        fig.suptitle(title)

In [10]:
from tensorflow.keras.applications.vgg16 import VGG16

model = VGG16(weights="imagenet", 
              include_top=False, 
              input_shape=X_train_rgb.shape[1:])

2022-03-03 20:20:17.845743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-03 20:20:17.949704: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-03 20:20:17.950072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-03 20:20:17.951945: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [11]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [12]:
model.trainable = False
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [13]:
flatten_layer = Flatten()
dense_layer = Dense(200, activation='relu')
prediction_layer = Dense(7, activation='softmax')

new_model = Sequential([model,flatten_layer,dense_layer,prediction_layer])

In [14]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 200)               102600    
                                                                 
 dense_1 (Dense)             (None, 7)                 1407      
                                                                 
Total params: 14,818,695
Trainable params: 104,007
Non-trainable params: 14,714,688
_________________________________________________________________


to be continued...

In [15]:
new_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
es = EarlyStopping(patience=3, restore_best_weights=True)

history = new_model.fit(X_train_rgb, y_train_cat,
                    epochs=50,  # Use early stop in practice
                    validation_data=(X_val_rgb,y_val_cat),
                    use_multiprocessing = True,
                    callbacks=[es],
                    batch_size=32, 
                    verbose=1)

2022-03-03 20:20:20.715040: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 793746432 exceeds 10% of free system memory.
2022-03-03 20:20:21.597968: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 793746432 exceeds 10% of free system memory.


Epoch 1/50


2022-03-03 20:20:24.368726: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8302
2022-03-03 20:20:26.572039: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 662.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-03 20:20:26.651002: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 637.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-03 20:20:26.667437: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.21GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more me

898/898 [==============================] - ETA: 0s - loss: 1.6193 - accuracy: 0.3651

2022-03-03 20:24:34.536058: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 575.94MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-03 20:24:34.593069: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 606.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-03 20:24:34.605386: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.10GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-03 20:24:34.670238: W tensorflow/core/common_runtime/bfc_allocator.cc:275] A

898/898 [==============================] - 275s 301ms/step - loss: 1.6193 - accuracy: 0.3651 - val_loss: 1.5467 - val_accuracy: 0.4110
Epoch 2/50
898/898 [==============================] - 155s 172ms/step - loss: 1.5302 - accuracy: 0.4094 - val_loss: 1.5166 - val_accuracy: 0.4168
Epoch 3/50
898/898 [==============================] - 143s 159ms/step - loss: 1.4878 - accuracy: 0.4268 - val_loss: 1.5124 - val_accuracy: 0.4302
Epoch 4/50
898/898 [==============================] - 135s 150ms/step - loss: 1.4486 - accuracy: 0.4450 - val_loss: 1.4812 - val_accuracy: 0.4363
Epoch 5/50
898/898 [==============================] - 129s 144ms/step - loss: 1.4141 - accuracy: 0.4609 - val_loss: 1.4735 - val_accuracy: 0.4388
Epoch 6/50
898/898 [==============================] - 120s 133ms/step - loss: 1.3797 - accuracy: 0.4762 - val_loss: 1.4747 - val_accuracy: 0.4386
Epoch 7/50
898/898 [==============================] - 116s 130ms/step - loss: 1.3439 - accuracy: 0.4935 - val_loss: 1.4930 - val_accura